**MVUM and Water Proximity Legality Rules**

In national forests, dispersed camping is typically allowed within 300 feet of an MVUM road, but not within 300 feet of water. There are exceptions, but we can use this rule as a rough guide to see where camping might be legal or is definitely illegal. First, let's set up paths and import helpers.

In [1]:
%run ../../bootstrap.py
setup_project_path()

from scripts.io_helpers import read_interim_layer, export_interim
from scripts.data_helpers import to_buffer_crs
from scripts.legality_helpers import tag_legality, merge_legality_layers
from scripts import data_config as dc

import geopandas as gpd
import pandas as pd

**Loading Data**

For this rule, we need water data (including flowlines and water areas) and MVUM roads. Since this rule is specific to national forests, let's load the pre-filtered data for national forests (make sure to run the filter_to_nf.ipynb notebook first).

In [ ]:
# Load and project dataset
flowline = read_interim_layer("water_flowline_nf")
flowline = to_buffer_crs(flowline)

water_area = read_interim_layer("water_polygon_nf")
water_area = to_buffer_crs(water_area)

mvum = read_interim_layer("mvum_nf")
mvum = to_buffer_crs(mvum)

Now, we're ready to create legality rules. To start, we'll assume that buffered mvum geometry is likely or maybe legal for camping. Then, we'll do an erasure between the buffered mvum land and buffered water data to get rid of illegal near-water camping. We'll also add the buffered water layer geometry to an illegal layer.

First, let's import the legality schema.

In [3]:
legality_schema = read_interim_layer("legality_schema")
legality_schema.head()

,legality,reasoning,land_owner,last_updated,geometry


Next, let's combine water data into a single geodataframe for simplicity.

In [4]:
water_nf = gpd.GeoDataFrame(
    pd.concat([water_area, flowline], ignore_index=True),
    crs=water_area.crs
)

water_nf.head(1)

,Mang_Name,Loc_Mang,Unit_Nm,Loc_Nm,Des_Tp,Pub_Access,Own_Name,State_Nm,Shape_Length,Shape_Area,gnis_name,fcode,flowdir,innetwork,nhdplusid,lengthkm,reachcode,NHDType,geometry
0,USFS,Forest Service Region 02 Rocky Mountain,Arapaho National Forest,Arapaho and Roosevelt National Forests,NF,OA,USFS,CO,2.344493e+06,2.391679e+09,None,46007,1,1,2.300180e+13,0.393332,10180001016862,NetworkNHDFlowline,"MULTIPOLYGON (((420036.616 4470912.486, 420054..."


**Legality Rules**

First, let's mark areas in the buffered water geometry as illegal. We'll make a copy of the buffered water data, add columns for the legality schema, then filter to the legality columns.

We can use the tag_legality helper from scripts/legality_helpers.py, which handles all the logic. We just need to pass the gdf with geometry as well as the legality parameters (legality, reasoning, etc.).

In [5]:
illegal_water = tag_legality(
    gdf=water_nf,
    legality="illegal",
    reasoning="Within 200 feet of water features",
    land_owner="USFS"
)

illegal_water.head()

,legality,reasoning,land_owner,last_updated,geometry
0,illegal,Within 200 feet of water features,USFS,2025-08-08,"MULTIPOLYGON (((420036.616 4470912.486, 420054..."
1,illegal,Within 200 feet of water features,USFS,2025-08-08,"MULTIPOLYGON (((412986.51 4469533.494, 413007...."
2,illegal,Within 200 feet of water features,USFS,2025-08-08,"MULTIPOLYGON (((419676.607 4470505.485, 419686..."
3,illegal,Within 200 feet of water features,USFS,2025-08-08,"MULTIPOLYGON (((441209.066 4407825.48, 441325...."
4,illegal,Within 200 feet of water features,USFS,2025-08-08,"MULTIPOLYGON (((439572.415 4408748.563, 439576..."


Now, let's create the likely legal layer. We'll start with buffered mvum, and do an erasure with buffered water. Then, we'll tag legality and merge that layer with the illegal layer we just generated.

In [6]:
mvum_erasure = gpd.overlay(mvum, water_nf, how="difference")

likely_legal = tag_legality(
    mvum_erasure, 
    legality="likely legal", 
    reasoning="within 300 feet of mvum access, sufficient distance from water", 
    land_owner="USFS"
)

likely_legal.head()

/Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/venv/lib/python3.11/site-packages/geopandas/tools/overlay.py:357: UserWarning: `keep_geom_type=True` in overlay resulted in 8176 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  result = _collection_extract(result, geom_type, keep_geom_type_warning)


,legality,reasoning,land_owner,last_updated,geometry
0,likely legal,"within 300 feet of mvum access, sufficient dis...",USFS,2025-08-08,"MULTIPOLYGON (((405155.406 4466766.474, 405169..."
1,likely legal,"within 300 feet of mvum access, sufficient dis...",USFS,2025-08-08,"MULTIPOLYGON (((403041.513 4449536.861, 403035..."
2,likely legal,"within 300 feet of mvum access, sufficient dis...",USFS,2025-08-08,"MULTIPOLYGON (((446385.014 4412202.779, 446388..."
3,likely legal,"within 300 feet of mvum access, sufficient dis...",USFS,2025-08-08,"POLYGON ((451380.233 4413886.949, 451381.143 4..."
4,likely legal,"within 300 feet of mvum access, sufficient dis...",USFS,2025-08-08,"MULTIPOLYGON (((447595.135 4408908.129, 447596..."


Now, let's merge those two legality layers using the merge_legality_layers helper.

In [7]:
nf_water_mvum_legality = merge_legality_layers([illegal_water, likely_legal])
export_interim(nf_water_mvum_legality, "nf_water_mvum_legality")

Saved to interim: /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/data/interim/nf_water_mvum_legality.gpkg
Also saved to processed: /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/data/processed/nf_water_mvum_legality.gpkg
